<a href="https://colab.research.google.com/github/jh17sh21-oss/gendered-racism-dataset/blob/main/Topic(Tf_IDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ===== 0) 설치 & 임포트 =====
!pip install -q konlpy==0.6.0 JPype1==1.5.0 scikit-learn pandas openpyxl

import re
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# ===== 1) 경로/열 설정 =====
file_path = 'topic_rawdata.xlsx'   # 업로드한 파일명/경로
sheet_name = '시트1'
TEXT_COL = '원자료'

# ===== 2) 불용어 & 표준화 사전 =====
stopwords = set([
    '그','이','저','것','등','때','중','누구','무엇','하다','되다','있다','없다','받다',
    '가다','오다','보다','주다','말하다','들다','계속','이제','지금','예전','요즘','거의',
    '좀','많이','더','다시','별로','그리고','그래서','그런데','하지만','그러니까','때문',
    '이런','그런','어떤','같다','나','내','너','우리','당신','그녀','그들','사람','모두',
    '여기','저기','거기','안','밖','위','아래','그곳','방향','도','는','만','과','와',
    '보다','까지','부터','으로','에게','이랑','밖에','조차','정도','경우','내용','문제',
    '이야기','모습','상황','자신','음','응','오','어휴','허','아이고','에휴','웃음',
    '그떄','그다음','아마','...','…','“','”','‘','’','-','--','―','그냥','진짜','완전',
    '약간','어떻게','뭐','딱','막','또','또는','해도','생각','중략','가지','조금',
    '자기','그때','그거','수도','그게','여러','무슨','네네','나이','어디','먼저','대부분',
    '나중','대해','그것','뭔가','전혀','저희','만약','이주','나가야','다른','항상',
    '얘기','나라','부분','선생님', '보고','사실','한국', '중국', '베트남', '일본',
    # 대화체 흔어(필요시 더 추가 가능)
    '이렇게','제가','근데','그렇게','저는','하는','있어요','하고','같아요','있는','있고',
    '얘기를','되게','그러면','이거','너무','때문에','같은','왜냐하면','하면','같이','때는',
    '사람들이','거는','것도','저한테','아니면','해요','내가','있는데','있잖아요'
])

standardization_dict = {
    "아빠":"남편","여자":"여성","아기":"아이","한국말":"한국어",
    "시엄마":"시어머니","어머니":"시어머니","대한민국":"한국","여자":"여성", "남자":"남성",
    "외국":"외국인","외국사람":"외국인","차별적":"차별","무시당함":"무시",
    "무시함":"무시","편견들":"편견","회사":"직장",
    "일":"직장","한국사람":"한국인",
    # 자녀 계열 통합
    "자녀":"아이","아들":"아이","딸":"아이","자식":"아이",
}

# ===== 3) 전처리: 명사만 추출 → 표준화 → 불용어/한글자 제거 =====
okt = Okt()

def normalize_token(token: str) -> str:
    return standardization_dict.get(token, token)

def nouns_only(text: str):
    text = str(text)
    nouns = okt.nouns(text)  # 명사만 추출
    # 표준화 사전 적용
    nouns = [normalize_token(n) for n in nouns]
    # 불용어/한글자 제거
    nouns = [n for n in nouns if len(n) > 1 and n not in stopwords]
    return nouns

# ===== 4) 데이터 로드 & 코퍼스 생성 =====
raw = pd.read_excel(file_path, sheet_name=sheet_name)
tokens_per_doc = raw[TEXT_COL].astype(str).apply(nouns_only)
docs = tokens_per_doc.apply(lambda toks: " ".join(toks)).tolist()

# ===== 5) TF-IDF (명사 토큰 전용) 상위 20 =====
vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b",
                      lowercase=False,
                      max_df=0.95,
                      sublinear_tf=True)
X = vec.fit_transform(docs)
terms = vec.get_feature_names_out()
df_count = (X > 0).sum(axis=0).A1
tfidf_mean = X.mean(axis=0).A1

res = pd.DataFrame({'키워드': terms, 'n': df_count, 'TF-IDF': tfidf_mean})
top20 = res.sort_values('TF-IDF', ascending=False).head(20).reset_index(drop=True)

print(top20)
top20.to_csv('tfidf_overall_top20_nouns.csv', index=False, encoding='utf-8-sig')

    키워드    n    TF-IDF
0    아이  102  0.044712
1    남편   89  0.042580
2   외국인  124  0.040624
3    직장  122  0.040598
4    여성   76  0.034911
5    엄마   61  0.034827
6    차별   88  0.031756
7    결혼   51  0.027208
8   한국어   62  0.026579
9    남성   50  0.026322
10   친구   52  0.022850
11   처음   65  0.020438
12   마음   57  0.019859
13  한국인   54  0.019278
14   느낌   46  0.018359
15   질문   27  0.017823
16   가끔   33  0.017745
17  다문화   29  0.017204
18   기분   32  0.015876
19   영어   28  0.015601
